# LLM Function Calling - MCP Server

Learn how to build **MCP (Model Context Protocol) servers** for reusable tool integration.

**What you'll learn:**
- What MCP is and why it's useful
- Build an MCP server using fastMCP
- Convert vanilla function calling to MCP tools
- Understand server/client architecture

**Use Case:** Same 18 Git commands, now as a reusable MCP server

## What is MCP (Model Context Protocol)?

**MCP** is a standardized protocol for connecting LLM agents with tools and data sources.

### Why MCP Exists

In Notebook 02, we built a **vanilla agent** where:
- Tool schemas were manually defined in JSON
- The LLM and tools were tightly coupled in one script
- Tools couldn't be reused by other agents or frameworks

**MCP solves this** by introducing a **server/client architecture:**

```
┌─────────────────┐         ┌──────────────────┐
│   MCP Client    │ ◄─────► │   MCP Server     │
│  (DSPy, LG)     │   MCP   │  (Git Tools)     │
│                 │Protocol │                  │
└─────────────────┘         └──────────────────┘
```

### Key Benefits

1. **Reusability**: One server, many clients (DSPy, LangGraph, Claude Desktop, etc.)
2. **Standardization**: Tools follow a common protocol (no custom formats)
3. **Separation**: Server code stays separate from agent logic
4. **Interoperability**: Any MCP client can use any MCP server

### MCP vs Vanilla

| Aspect          | Vanilla (Notebook 02)            | MCP Server (This Notebook)     |
|-----------------|----------------------------------|--------------------------------|
| **Coupling**    | Tight (everything in one script) | Loose (server/client separate) |
| **Reusability** | Single use                       | Multiple clients can connect   |
| **Schema**      | Manual JSON                      | Auto-generated from code       |
| **Protocol**    | Custom                           | Standardized (MCP)             |

In this notebook, we'll convert our 18 Git tools to an MCP server using **fastMCP**.

![Model Context Protocol](../assets/mcp.gif)

## Setup

We'll use **fastMCP** - a Python library for building MCP servers.

**fastMCP Features:**
- Decorator-based tool registration (`@mcp.tool()`)
- Automatic schema generation from type hints
- Pythonic and minimal boilerplate
- Handles MCP protocol details automatically

In [ ]:
# Install fastMCP
!pip install fastmcp

In [1]:
import subprocess
from typing import List, Optional
from fastmcp import FastMCP

print("✓ Imports successful")

✓ Imports successful


## Step 1: Initialize MCP Server

Create a FastMCP instance with a name for your server.

In [2]:
# Initialize MCP server
mcp = FastMCP("Git Assistant")

print("✓ MCP server initialized")

✓ MCP server initialized


## Step 2: Define Tools with Decorators

In Notebook 02, we manually wrote JSON schemas for each tool:

```python
# Vanilla approach (Notebook 02)
{
    "name": "git_status",
    "description": "Show git status",
    "parameters": {
        "type": "object",
        "properties": {},
        "required": []
    }
}
```

**With fastMCP**, we use decorators and Python type hints:

```python
# MCP approach (this notebook)
@mcp.tool()
def git_status() -> str:
    """Show git status"""
    # implementation
```

**How it works:**
- `@mcp.tool()` - Registers the function as an MCP tool
- Type hints (`str`, `List[str]`, etc.) - Auto-generates parameter schemas
- Docstring - Becomes the tool description for the LLM
- Return type - Specifies what the tool returns

Let's implement all 18 Git commands as MCP tools.

In [3]:
# Git Configuration & Commit Tools

@mcp.tool()
def git_config_username(username: str) -> str:
    """Configure Git global username"""
    result = subprocess.run(
        ["git", "config", "--global", "user.name", username],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        return f"✓ Username set to: {username}"
    return f"✗ Error: {result.stderr}"

@mcp.tool()
def git_config_email(email: str) -> str:
    """Configure Git global email"""
    result = subprocess.run(
        ["git", "config", "--global", "user.email", email],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        return f"✓ Email set to: {email}"
    return f"✗ Error: {result.stderr}"

@mcp.tool()
def git_commit(message: str) -> str:
    """Commit staged changes with a message"""
    result = subprocess.run(
        ["git", "commit", "-m", message],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        return f"✓ Committed: {message}\n{result.stdout}"
    return f"✗ Error: {result.stderr}"

print("✓ Registered 3 tools: git_config_username, git_config_email, git_commit")

✓ Registered 3 tools: git_config_username, git_config_email, git_commit


In [4]:
# Git Branch Operations

@mcp.tool()
def git_checkout(branch_name: str) -> str:
    """Switch to a specific branch"""
    result = subprocess.run(
        ["git", "checkout", branch_name],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        return f"✓ Switched to branch: {branch_name}\n{result.stdout}"
    return f"✗ Error: {result.stderr}"

@mcp.tool()
def git_create_branch(branch_name: str) -> str:
    """Create a new branch"""
    result = subprocess.run(
        ["git", "branch", branch_name],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        return f"✓ Created branch: {branch_name}"
    return f"✗ Error: {result.stderr}"

@mcp.tool()
def git_delete_branch(branch_name: str) -> str:
    """Delete a branch"""
    result = subprocess.run(
        ["git", "branch", "-d", branch_name],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        return f"✓ Deleted branch: {branch_name}"
    return f"✗ Error: {result.stderr}"

@mcp.tool()
def git_rename_branch(old_name: str, new_name: str) -> str:
    """Rename a branch"""
    result = subprocess.run(
        ["git", "branch", "-m", old_name, new_name],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        return f"✓ Renamed branch: {old_name} → {new_name}"
    return f"✗ Error: {result.stderr}"

print("✓ Registered 4 tools: git_checkout, git_create_branch, git_delete_branch, git_rename_branch")

✓ Registered 4 tools: git_checkout, git_create_branch, git_delete_branch, git_rename_branch


In [5]:
# Git Status & Reset Tools

@mcp.tool()
def git_status() -> str:
    """Show the current git status"""
    result = subprocess.run(
        ["git", "status"],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        return result.stdout
    return f"✗ Error: {result.stderr}"

@mcp.tool()
def git_reset_last_commit(mode: str) -> str:
    """Remove the last commit (mode: soft, mixed, or hard)"""
    if mode not in ["soft", "mixed", "hard"]:
        return "✗ Error: mode must be 'soft', 'mixed', or 'hard'"
    
    result = subprocess.run(
        ["git", "reset", f"--{mode}", "HEAD~1"],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        return f"✓ Reset last commit ({mode} mode)\n{result.stdout}"
    return f"✗ Error: {result.stderr}"

print("✓ Registered 2 tools: git_status, git_reset_last_commit")

✓ Registered 2 tools: git_status, git_reset_last_commit


In [6]:
# Git Remote Operations

@mcp.tool()
def git_add_remote(name: str, url: str) -> str:
    """Add a new remote repository"""
    result = subprocess.run(
        ["git", "remote", "add", name, url],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        return f"✓ Added remote: {name} → {url}"
    return f"✗ Error: {result.stderr}"

@mcp.tool()
def git_remove_remote(name: str) -> str:
    """Remove a remote repository"""
    result = subprocess.run(
        ["git", "remote", "remove", name],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        return f"✓ Removed remote: {name}"
    return f"✗ Error: {result.stderr}"

@mcp.tool()
def git_list_remotes() -> str:
    """List all configured remotes"""
    result = subprocess.run(
        ["git", "remote", "-v"],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        return result.stdout if result.stdout else "No remotes configured"
    return f"✗ Error: {result.stderr}"

print("✓ Registered 3 tools: git_add_remote, git_remove_remote, git_list_remotes")

✓ Registered 3 tools: git_add_remote, git_remove_remote, git_list_remotes


In [7]:
# Git Staging Operations

@mcp.tool()
def git_add(files: List[str]) -> str:
    """Add files to the staging area"""
    result = subprocess.run(
        ["git", "add"] + files,
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        return f"✓ Added to staging: {', '.join(files)}"
    return f"✗ Error: {result.stderr}"

@mcp.tool()
def git_unstage(files: List[str]) -> str:
    """Remove files from the staging area"""
    result = subprocess.run(
        ["git", "reset", "HEAD"] + files,
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        return f"✓ Unstaged: {', '.join(files)}"
    return f"✗ Error: {result.stderr}"

print("✓ Registered 2 tools: git_add, git_unstage")

✓ Registered 2 tools: git_add, git_unstage


In [8]:
# Git Pull & Push Operations

@mcp.tool()
def git_pull(remote: Optional[str] = "origin", branch: Optional[str] = "main") -> str:
    """Pull changes from a remote repository"""
    result = subprocess.run(
        ["git", "pull", remote, branch],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        return f"✓ Pulled from {remote}/{branch}\n{result.stdout}"
    return f"✗ Error: {result.stderr}"

@mcp.tool()
def git_push(remote: Optional[str] = "origin", branch: Optional[str] = "main") -> str:
    """Push changes to a remote repository"""
    result = subprocess.run(
        ["git", "push", remote, branch],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        return f"✓ Pushed to {remote}/{branch}\n{result.stdout}"
    return f"✗ Error: {result.stderr}"

print("✓ Registered 2 tools: git_pull, git_push")

✓ Registered 2 tools: git_pull, git_push


In [9]:
# Git Repository Initialization

@mcp.tool()
def git_init(directory: Optional[str] = ".") -> str:
    """Initialize a new git repository"""
    result = subprocess.run(
        ["git", "init", directory],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        return f"✓ Initialized git repository in: {directory}\n{result.stdout}"
    return f"✗ Error: {result.stderr}"

@mcp.tool()
def git_clone(url: str, directory: Optional[str] = None) -> str:
    """Clone a git repository"""
    cmd = ["git", "clone", url]
    if directory:
        cmd.append(directory)
    
    result = subprocess.run(
        cmd,
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        target = directory if directory else url.split('/')[-1].replace('.git', '')
        return f"✓ Cloned repository to: {target}\n{result.stdout}"
    return f"✗ Error: {result.stderr}"

print("✓ Registered 2 tools: git_init, git_clone")
print(f"\n🎉 Total: 18 Git tools registered with MCP server!")

✓ Registered 2 tools: git_init, git_clone

🎉 Total: 18 Git tools registered with MCP server!


## Step 3: Inspect the Server

Let's examine what fastMCP automatically generated from our function definitions.

In [15]:
# List all registered tools
tool_names = await mcp.get_tools()

print("=" * 60)
print(f"MCP Server: {mcp.name}")
print("=" * 60)
print(f"\nTotal tools registered: {len(tool_names)}\n")
print(tool_names)

MCP Server: Git Assistant

Total tools registered: 18

{'git_config_username': FunctionTool(name='git_config_username', title=None, description='Configure Git global username', tags=set(), enabled=True), 'git_config_email': FunctionTool(name='git_config_email', title=None, description='Configure Git global email', tags=set(), enabled=True), 'git_commit': FunctionTool(name='git_commit', title=None, description='Commit staged changes with a message', tags=set(), enabled=True), 'git_checkout': FunctionTool(name='git_checkout', title=None, description='Switch to a specific branch', tags=set(), enabled=True), 'git_create_branch': FunctionTool(name='git_create_branch', title=None, description='Create a new branch', tags=set(), enabled=True), 'git_delete_branch': FunctionTool(name='git_delete_branch', title=None, description='Delete a branch', tags=set(), enabled=True), 'git_rename_branch': FunctionTool(name='git_rename_branch', title=None, description='Rename a branch', tags=set(), enabled=T

## Step 4: Running the MCP Server

### In a Production Environment

To run this MCP server in a real application, you would:

1. **Save to a Python file** (e.g., `git_server.py`)
2. **Add the run block** at the end:

```python
if __name__ == "__main__":
    mcp.run()
```

3. **Start the server**:
```bash
python git_server.py
```

The server will then:
- Listen for MCP protocol messages
- Accept connections from MCP clients
- Execute tools when requested
- Return results in standardized format

### In This Notebook

Since we're in a Jupyter notebook environment, we've **defined** the server but haven't started it. The server definition is complete and ready to use!

**Next Steps:**
- **Notebook 05**: We'll create a DSPy MCP client to connect to this server
- **Notebook 06**: We'll create a LangGraph MCP client to connect to this server

Both clients will be able to use these same 18 Git tools through the MCP protocol.

## Comparison: Vanilla vs MCP Server

| Aspect                | Vanilla (Notebook 02) | MCP Server (This Notebook)             |
|-----------------------|-----------------------|----------------------------------------|
| **Tool Definition**   | Manual JSON schemas   | Decorators + type hints                |
| **Schema Generation** | Manual (error-prone)  | Automatic from code                    |
| **Protocol**          | Custom/ad-hoc         | Standardized (MCP)                     |
| **Reusability**       | Single script only    | Any MCP client can use                 |
| **Client Coupling**   | Tight (same codebase) | Loose (protocol-based)                 |
| **Framework Support** | None                  | DSPy, LangGraph, Claude Desktop        |
| **Boilerplate**       | High (JSON schemas)   | Low (decorators)                       |
| **Type Safety**       | Manual validation     | Automatic from type hints              |
| **Maintenance**       | Update schema + code  | Update code only (schema auto-updates) |